In [97]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [98]:
import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far

In [99]:
far.utils._check()

4


In [100]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
# v1 = tf.Variable(1.)

v2 = tf.Variable([-1.])

In [136]:
def _test(method=far.ForwardHG):
    try: ss.close()
    except: pass
    tf.reset_default_graph()
    ss = tf.InteractiveSession()
    v1 = tf.Variable([[1., -1., 1.], [2., 10., 2.]])

    v2 = tf.Variable([-1., 1., 0.2])
    lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape(), scalar=True)

    cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)
#     cost = tf.reduce_sum(lmbd*v2**2)

    
    
    _et = far.get_hyperparameter('eta', 0.1)
    io_optim = far.MomentumOptimizer(_et, far.get_hyperparameter('mu', .5))
    # io_optim = far.GradientDescentOptimizer(_et)

    io_optim_dict = io_optim.minimize(cost) 
    oo = tf.reduce_mean(v1*v2)
#     oo = tf.reduce_mean(v2)
    optim_oo = tf.train.AdamOptimizer(.01)
    farho = far.HyperOptimizer(hypergradient=method())
    farho.minimize(oo, optim_oo, cost, io_optim)
    return ss, farho, cost, oo

In [137]:
ss, farho, cost, oo = _test()

[<tf.Tensor 'Mean_1_1/gradients_4/Mean_1_1/DUMMY/Vectorization/Reshape_grad/Reshape:0' shape=(2, 3) dtype=float32>, <tf.Tensor 'Mean_1_1/gradients_4/Mean_1_1/DUMMY/Vectorization/Reshape_1_grad/Reshape:0' shape=(2, 3) dtype=float32>, <tf.Tensor 'Mean_1_1/gradients_4/Mean_1_1/DUMMY/Vectorization/Reshape_2_grad/Reshape:0' shape=(3,) dtype=float32>, <tf.Tensor 'Mean_1_1/gradients_4/Mean_1_1/DUMMY/Vectorization/Reshape_3_grad/Reshape:0' shape=(3,) dtype=float32>]
[<tf.Tensor 'Mean_1_1/Dot_1/Sum:0' shape=() dtype=float32>, <tf.Tensor 'Mean_1_1/Dot_2/Sum:0' shape=() dtype=float32>]
None
[<tf.Tensor 'Mean_1_1/gradients_8/Mean_1_1/DUMMY/Vectorization/Reshape_grad/Reshape:0' shape=(2, 3) dtype=float32>, <tf.Tensor 'Mean_1_1/gradients_8/Mean_1_1/DUMMY/Vectorization/Reshape_1_grad/Reshape:0' shape=(2, 3) dtype=float32>, <tf.Tensor 'Mean_1_1/gradients_8/Mean_1_1/DUMMY/Vectorization/Reshape_2_grad/Reshape:0' shape=(3,) dtype=float32>, <tf.Tensor 'Mean_1_1/gradients_8/Mean_1_1/DUMMY/Vectorization/Res

In [138]:
tf.global_variables_initializer().run()

In [139]:
# execution with gradient descent! (looks ol)
farho.run(10, _skip_hyper_ts=True)

ss.run(far.utils.hypergradients())

[-0.018393533, 0.055180617, 0.003678706, 0.53872275, -0.076542258]

In [87]:
zs = tf.get_collection(far.GraphKeys.ZS)

In [88]:
ss.run([z.assign(tf.ones_like(z)) for z in zs])

[array([ 1.], dtype=float32),
 array([ 1.], dtype=float32),
 array([ 1.], dtype=float32),
 array([ 1.], dtype=float32),
 array([ 1.], dtype=float32),
 array([ 1.], dtype=float32)]

In [89]:
od = list(farho.hypergradient._optimizer_dicts)[0]
od

In [90]:
dyn_d_z = far.dot(far.vectorize_all(zs[:2]), far.vectorize_all(od.dynamics))

In [91]:
grds = tf.gradients(dyn_d_z, od.state)

In [92]:
ss.run(grds)

[array([ 2.79999995], dtype=float32), array([ 0.44999999], dtype=float32)]

In [83]:
0.8 - 0.1*0.8

0.72

In [95]:
- 0.1*0.5 + 0.5

0.45

In [96]:
1 - 0.1*2 + 2

2.8

In [52]:
dynamics = od.dynamics

In [ ]:
a_matrix = [[
        tf.gradients
    ]]

In [64]:
od.state

[<tf.Variable 'Variable:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'Variable/Momentum:0' shape=(1,) dtype=float32_ref>]

In [43]:
a_d_z = farho.hypergradient.A_dot_zs[far.utils.hyperparameters()[0]]

In [45]:
ss.run(a_d_z)

[array([ 2.79999995], dtype=float32), array([ 0.80999994], dtype=float32)]

In [47]:
ss.run(far.utils.hyperparameters())

[1.0, 0.1, 0.89999998]

In [24]:
# ss.run(tf.get_collection(far.GraphKeys.ZS))

In [104]:
# execution with gradient descent! (looks ol)
farho.run(10, _skip_hyper_ts=True)

ss.run(far.utils.hypergradients())

[-0.070845306, -0.70845306, 0.17504753]

In [26]:
# # execution with gradient descent! (looks ol)
# run(1, _skip_hyper_ts=True)

# ss.run(far.utils.hypergradients())

In [105]:
ss.run(cost), ss.run(oo)

(0.001059196, 0.032545291)

# CHECK WITH REVERSE_HG

In [140]:
ss, farho, cost, oo = _test(far.ReverseHg)

In [141]:
tf.global_variables_initializer().run()

In [142]:
# execution with gradient descent! (looks ol)
farho.run(10, _skip_hyper_ts=True)

ss.run(far.utils.hypergradients())

[-0.018393531, 0.055180617, 0.0036787055, 0.53872281, -0.076542258]

In [ ]:
[0.0023056811, 0.046113621, -0.011393946]

In [ ]:
[0.00076856039, -0.00076856039, -0.00015371206, -0.0030742411, 0.00075959647]

In [ ]:
[-0.018393533, 0.055180617, 0.003678706, 0.53872275, -0.076542258]

In [29]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable(1.)

v2 = tf.Variable([-1.])

In [73]:
lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape(), scalar=True)

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

_et = far.get_hyperparameter('eta', 0.1)
io_optim = far.MomentumOptimizer(_et, far.get_hyperparameter('mu', .9))
# io_optim = far.GradientDescentOptimizer(_et)


io_optim_dict = io_optim.minimize(cost) 

oo = tf.reduce_mean(v1*v2)

optim_oo = tf.train.AdamOptimizer(.01)

In [74]:
farho = far.HyperOptimizer()
run = farho.minimize(oo, optim_oo, cost, io_optim)

In [75]:
tf.global_variables_initializer().run()

In [76]:
# execution with gradient descent! (looks ol)
run(10, _skip_hyper_ts=True)

ss.run(far.utils.hypergradients())

[-0.0066759503, -0.13351899, -0.0026351726]

In [77]:
ss.run(cost), ss.run(oo)

(3.8717641e-05, -1.935882e-05)

In [46]:
# execution with gradient descent! (looks ol)
run(10, _skip_hyper_ts=True)

ss.run(far.utils.hypergradients())

[0.028823035, 0.57646066]

In [78]:
run(1, _skip_hyper_ts=True)

ss.run(far.utils.hypergradients())

[0.16000001, 3.2, 0.0]

In [ ]:
[0.023399413, 0.14039648, -0.023399413, 0.0, 2.0279489]